<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marcoteran/ml/blob/master/notebooks/ml_svm_modelselection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir en Colab" title="Abrir y ejecutar en Google Colaboratory"/></a>
  </td>
  <td>
    <a target="_blank" href="https://kaggle.com/kernels/welcome?src=https://github.com/marcoteran/ml/blob/master/notebooks/ml_svm_modelselection.ipynb"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Abrir en Kaggle" title="Abrir y ejecutar en Kaggle"/></a>
  </td>
</table>

# Sesión 04: Máquinas de vectores de soporte y selección de modelos
## Guía Completa

**Machine Learning**

**Profesor:** Marco Terán  
**Fecha:** 2025

[Website](http://marcoteran.github.io/),
[Github](https://github.com/marcoteran),
[LinkedIn](https://www.linkedin.com/in/marcoteran/).
___

Definimos primero unas librerías y funciones que vamos a usar a durante la sesión:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pylab as pl
from sklearn.model_selection import train_test_split
from matplotlib.colors import Normalize

In [ ]:
# Función para visualizar un conjunto de datos en 2D
def plot_data(X, y):
    y_unique = np.unique(y)
    colors = pl.cm.rainbow(np.linspace(0.0, 1.0, y_unique.size))
    for this_y, color in zip(y_unique, colors):
        this_X = X[y == this_y]
        pl.scatter(this_X[:, 0], this_X[:, 1],  c=color.reshape(1,-1),
                    alpha=0.5, edgecolor='k',
                    label="Class %s" % this_y)
    pl.legend(loc="best")
    pl.title("Data")
    
# Función para visualizar de la superficie de decisión de un clasificador
def plot_decision_region(X, pred_fun):
    min_x = np.min(X[:, 0])
    max_x = np.max(X[:, 0])
    min_y = np.min(X[:, 1])
    max_y = np.max(X[:, 1])
    min_x = min_x - (max_x - min_x) * 0.05
    max_x = max_x + (max_x - min_x) * 0.05
    min_y = min_y - (max_y - min_y) * 0.05
    max_y = max_y + (max_y - min_y) * 0.05
    x_vals = np.linspace(min_x, max_x, 100)
    y_vals = np.linspace(min_y, max_y, 100)
    XX, YY = np.meshgrid(x_vals, y_vals)
    grid_r, grid_c = XX.shape
    ZZ = np.zeros((grid_r, grid_c))
    for i in range(grid_r):
        for j in range(grid_c):
            ZZ[i, j] = pred_fun(XX[i, j], YY[i, j])
    pl.contourf(XX, YY, ZZ, 100, cmap = pl.cm.coolwarm, vmin= -1, vmax=2)
    pl.colorbar()
    pl.xlabel("x")
    pl.ylabel("y")
    
class MidpointNormalize(Normalize):
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y))
    
def gen_pred_fun(clf):
    def pred_fun(x1, x2):
        x = np.array([[x1, x2]])
        return clf.predict(x)[0]
    return pred_fun

def plot_labels(n_folds, n_classes, list_labels):
    ind = np.arange(n_folds)
    width = 0.15
    
    countings = []
    for labels in list_labels:
        labels = np.array(labels)
        countings.append([np.count_nonzero(labels == x) for x in range(n_classes)])
    
    class_bars = []
    for cls in range(n_classes):
        class_bars.append([l[cls] for l in countings])
    
    fig, ax = pl.subplots()
    i = 0
    for class_bar in class_bars:
        ax.bar(ind + width*i, class_bar, width, label='Clase '+str(i))
        i += 1
        
    ax.set_xticks(ind + 2*width / 3)
    ax.set_xticklabels(['Pliegue {}'.format(k) for k in range(n_folds)])
    pl.legend(loc="best")
    pl.title("Etiquetas")

# Máquinas de vectores de soporte
**Support Vector Machines**

Las máquinas de vectores de soporte (SVM)  son un conjunto de modelos de aprendizaje supervisado que se utilizan para la clasificación, la regresión y la detección de valores atípicos, en el cual los ejemplos son representados en un nuevo espacio, de tal forma que aquellos ejemplos de diferentes categorías sea posible, en principio, separarlos linealmente.

Las ventajas de las máquinas de vectores soporte son:
* Son eficaces en espacios de alta dimensión.
* Siguen siendo eficaces en los casos en que el número de dimensiones es mayor que el número de muestras.
* Utiliza un subconjunto de puntos de entrenamiento en la función de decisión (llamados vectores de soporte), por lo que también es eficiente en cuanto a memoria.
* **Versátil:** se pueden especificar diferentes funciones del Kernel para la función de decisión. Se proporcionan kernels comunes, pero también es posible especificar kernels personalizados.

Las desventajas de las máquinas de vectores de soporte son:
* Si el número de características es mucho mayor que el número de muestras, es crucial evitar el sobreajuste en la elección de las funciones Kernel y el término de regularización.
* Las SVM no proporcionan directamente estimaciones de probabilidad, éstas se calculan mediante una costosa validación cruzada de cinco pliegues.

## Definición del conjunto de datos

Vamos a trabajar con un conjunto de datos artificial (conjunto de datos de juguete). El conjunto es creado usando la funcionalidad `make_circles` de Scikit-Learn [ver más](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_circles.html).

`make_circles` permite crear un círculo grande que contenga un círculo más pequeño en 2d.
Un simple conjunto de datos de juguete para visualizar los algoritmos de agrupación y xclasificación.

Considere el siguiente ejemplo de un conjunto de datos:

In [ ]:
# Cargue las funciones datos make_circles n_samples=1000, factor=.3 (separación entre circulos), noise=.05
from sklearn.datasets import make_circles

In [ ]:
np.random.seed(0)
X, y = make_circles(n_samples=1000, factor=0.3, noise=0.05)

Se trata de un conjunto de datos que no es linealmente separable

In [ ]:
# Dibuje los datos
pl.figure(figsize=(10,6))
plot_data(X, y)

## ¿Cómo separar los datos?

SVM  crea, implícitamente, un espacio de representación de mayor dimensionalidad en el cual podemos separar de forma clara nuestros datos. Por ejemplo:

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/kernel_trick.png" width="60%">

## Kernel trick

SVM usa una función conocida como kernel. Intuitivamente, esta función $k$ define qué tan parecidas son dos instancias del conjunto de datos. Formalmente, la función $k$ calcula el producto punto en el espacio de caracterísiticas donde se representarán los datos. Dependiendo del kernel, este espacio de característica es de mayor dimensionalidad, y facilita la  definición de un "*hiperplano*" que separe los ejemplos de ambas características. 

Existen varias opciones para las funciones de kernel. Primero vamos a cargar dos conjuntos de datos sobre los cuales vamos a comparar la superficie de decisión generada por cada tipo de kernel.

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/differentkernels.png" width="60%">

In [ ]:
# Cargar el conjunto de datos de iris (clases: versicolor y virginica)
from sklearn.datasets import load_iris

In [ ]:
iris=load_iris()

In [ ]:
iris.target_names

In [ ]:
X=iris.data[:,[1,2]]

In [ ]:
y=iris.target

In [ ]:
# Dividir el conjunto en 70% para entrenamiento y 30% para prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [ ]:
# Cargar el conjunto de datos de moon: 600 muestras y noise 0.3
from sklearn.datasets import make_moons

X_moons, y_moons = make_moons(n_samples=600, noise=0.3, random_state=0)

In [ ]:
# Dividir el conjunto en 70% para entrenamiento y 30% para prueba
X_moons_train, X_moons_test, y_moons_train, y_moons_test = train_test_split(X_moons, y_moons, test_size=0.3, random_state=1)

In [ ]:
# Dibujar iris
# Dibuje los datos
pl.figure(figsize=(10,6))
plot_data(X, y)

In [ ]:
# Dibujar moon
# Dibuje los datos
pl.figure(figsize=(10,6))
plot_data(X_moons, y_moons)

## Kernel Lineal

La función $k$ está definida como:
$$
k(x,y) = \langle x, x\rangle = xx'
$$

Esta implementación puede ser consultada a través de `sklearn.svm.LinearSVC` [(ver más)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html).

In [ ]:
# Importamos el metodo
from sklearn.svm import LinearSVC

In [ ]:
# Crear dos clasificadores de kernel lineal: linear_iris y linear_moons
linear_iris=LinearSVC()
linear_moons=LinearSVC()

In [ ]:
# Entrenamos los modelo `LinearSVC` llamando la función `fit()` sobre el conjunto de datos reducido
linear_iris.fit(X_train, y_train)
linear_moons.fit(X_moons_train, y_moons_train)

Ahora visualizamos los datos de ambos y la superficie de decisión:

In [ ]:
pl.figure(figsize=(10,6))
plot_decision_region(X_test, gen_pred_fun(linear_iris))
plot_data(X_test, y_test)

El error en el conjunto de entrenamiento y prueba es el siguiente:

In [ ]:
# Error en entrenamiento para iris
# Error en prueba para iris
print('Error en entrenamiento para iris: {}'.format(1-linear_iris.score(X_train, y_train)))
print('Error en prueba para iris: {}'.format(1-linear_iris.score(X_test, y_test)))

Ahora hacemos los mismo para el conjunto de datos generado artificialmente:

In [ ]:
pl.figure(figsize=(10,6))
plot_decision_region(X_moons_test, gen_pred_fun(linear_iris))
plot_data(X_moons_test, y_moons_test)

In [ ]:
# Error en entrenamiento para moons
# Error en prueba para moons
print('Error en entrenamiento para moons: {}'.format(1-linear_iris.score(X_moons_train, y_moons_train)))
print('Error en prueba para moons: {}'.format(1-linear_iris.score(X_moons_test, y_moons_test)))

## Kernel polinomial

La función $k$ está definida como:
$$
k(x,y) = (\gamma \langle x, x'\rangle + r)^d
$$
dónde $d$ corresponde al grado del polinomio (parametro `degree`) y $r$ por `coef0`. De manera similar podemos acceder a la implementación de este kernel a través de `sklearn.svm.SVC` [(ver más)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

Definimos primero un kernel polinomial de grado $2$.

In [ ]:
# Importamos el metodo
from sklearn.svm import SVC

In [ ]:
# Definimos el clasificador con kernel='poly' y degree=2
poly_svm=SVC(kernel='poly',degree=2)

In [ ]:
poly_svm

In [ ]:
# Entrenamos los modelo `SVC` llamando la función `fit()` sobre el conjunto de datos reducido
poly_svm.fit(X_moons_train, y_moons_train)

Ahora visualizamos los datos de ambos y la superficie de decisión:

In [ ]:
pl.figure(figsize=(10,6))
plot_decision_region(X_moons_test, gen_pred_fun(poly_svm))
plot_data(X_moons_test, y_moons_test)

In [ ]:
# Error en entrenamiento para moons
# Error en prueba para moons
print('Error en entrenamiento para moons: {}'.format(1-poly_svm.score(X_moons_train, y_moons_train)))
print('Error en prueba para moons: {}'.format(1-poly_svm.score(X_moons_test, y_moons_test)))

Ahora lo intentamos con un polinomio de grado $3$:

In [ ]:
poly_svm_d3=SVC(kernel='poly',degree=3)
poly_svm_d3.fit(X_moons_train, y_moons_train)

Medimos el error en entrenamiento y prueba

In [ ]:
pl.figure(figsize=(10,6))
plot_decision_region(X_moons_test, gen_pred_fun(poly_svm_d3))
plot_data(X_moons_test, y_moons_test)

In [ ]:
print("Error en entrenamiento: {}".format(1-poly_svm_d3.score(X_moons_train, y_moons_train)))
print("Error en prueba: {}".format(1-poly_svm_d3.score(X_moons_test, y_moons_test)))

Ahora probamos con el conjunto de datos IRIS $(d=4)$:

In [ ]:
poly_svm = SVC(kernel='poly', degree=4)
poly_svm.fit(X_train, y_train)

pl.figure(figsize = (10, 6))    
plot_decision_region(X_test, gen_pred_fun(poly_svm))
plot_data(X_test, y_test)

In [ ]:
# Error en entrenamiento
# Error en prueba
print("Error en entrenamiento: {}".format(1-poly_svm.score(X_train, y_train)))
print("Error en prueba: {}".format(1-poly_svm.score(X_test, y_test)))

## Kernel Gaussiano


$$
K(x, x') = \exp\left(-\frac{\|x-x'\|^2}{2\sigma^2}\right)
$$
la cual se puede simplificar como
$$
K(x, x') = \exp(-\gamma \|x-x'\|^2)
$$

donde $\gamma$ se especifica mediante el parámetro `gamma`, que debe ser mayor que $0$.

En la literatura este método tambien se encuentra como kernel usando una función de base radial (**RBF** por sus siglas en ingles).

Al entrenar una SVM con el kernel de la función de base radial (RBF), hay que tener en cuenta dos parámetros: $C$ y $\gamma$.
* El parámetro $C$, común a todos los kernels de SVM, es el **parametro de regularización**, que compensa la clasificación errónea de las muestras de entrenamiento en la complejidad de la superficie de decisión.
    - Un $C$ bajo hace que la superficie de decisión sea suave, mientras que un $C$ alto tiene como objetivo clasificar correctamente todos los ejemplos de entrenamiento.

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/lowc.png" width="40%">
<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/highc.png" width="40%">

* $\gamma$ define cuánta influencia tiene una sola muestra de entrenamiento.
    - Cuanto mayor sea $\gamma$, más cerca deben estar las demás muestras para verse afectadas.

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/highgamma.png" width="40%">
<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/lowgamma.png" width="40%">

La elección adecuada de $C$ y $\gamma$ es fundamental para el rendimiento de la SVM. Se aconseja utilizar `GridSearchCV` con $C$ y $\gamma$ espaciados exponencialmente para elegir buenos valores.

En el siguiente ejemplo probamos con un valor de $\gamma$ pequeño $(0.007)$:

In [ ]:
# Cargamos el clasificador
rbf_svm = SVC(kernel='rbf', gamma=0.007)

In [ ]:
rbf_svm

In [ ]:
pl.figure(figsize = (10,6))
plot_data(X_moons_train, y_moons_train)

In [ ]:
# Entrenamos el clasificador
rbf_svm.fit(X_moons_train, y_moons_train)

Ahora visualizamos los datos de ambos y la superficie de decisión:

In [ ]:
pl.figure(figsize=(10,6))
plot_decision_region(X_moons_test, gen_pred_fun(rbf_svm))
plot_data(X_moons_test, y_moons_test)

Reportamos el error de entrenamiento y prueba:

In [ ]:
# Error en entrenamiento
# Error en prueba
print("Error en entrenamiento: {}".format(1-rbf_svm.score(X_moons_train, y_moons_train)))
print("Error en prueba: {}".format(1-rbf_svm.score(X_moons_test, y_moons_test)))

Usamos un $\gamma$ más grande $(10000)$

In [ ]:
rbf_svm = SVC(kernel='rbf', gamma=10000)
rbf_svm.fit(X_moons_train, y_moons_train)

In [ ]:
pl.figure(figsize=(10,6))
plot_decision_region(X_moons_test, gen_pred_fun(rbf_svm))
plot_data(X_moons_test, y_moons_test)

In [ ]:
pl.figure(figsize=(20,12))
plot_decision_region(X_moons_train, gen_pred_fun(rbf_svm))
plot_data(X_moons_train, y_moons_train)

Ahora reportamos el error de entrenamiento y prueba:

In [ ]:
# Error en entrenamiento
# Error en prueba
print("Error en entrenamiento: {}".format(1-rbf_svm.score(X_moons_train, y_moons_train)))
print("Error en prueba: {}".format(1-rbf_svm.score(X_moons_test, y_moons_test)))

Ahora escogemos un valor de $\gamma$ intermedio $(0.7)$:

In [ ]:
rbf_svm = SVC(kernel='rbf', gamma=0.7)
rbf_svm.fit(X_moons_train, y_moons_train)

pl.figure(figsize=(10,6))
plot_decision_region(X_moons_test, gen_pred_fun(rbf_svm))
plot_data(X_moons_test, y_moons_test)

In [ ]:
# Error en entrenamiento
# Error en prueba
print("Error en entrenamiento: {}".format(1-rbf_svm.score(X_moons_train, y_moons_train)))
print("Error en prueba: {}".format(1-rbf_svm.score(X_moons_test, y_moons_test)))

Probamos sobre el conjunto de datos IRIS $(\gamma=0.7)$

In [ ]:
rbf_svm = SVC(kernel='rbf', gamma=0.7)
rbf_svm.fit(X_train, y_train)

pl.figure(figsize=(10,6))
plot_decision_region(X_test, gen_pred_fun(rbf_svm))
plot_data(X_test, y_test)

Finalmente reportamos el error en entrenamiento y prueba:

In [ ]:
# Error en entrenamiento
# Error en prueba
print('Error en entrenamiento para iris: {}'.format(1-rbf_svm.score(X_train, y_train)))
print('Error en prueba para iris: {}'.format(1-rbf_svm.score(X_test, y_test)))

# Estimando de una forma más robusta los hiperparámetros del modelo

Hasta el momento nos hemos concentrado en evaluar nuestros modelos en una partición de prueba. Sin embargo, es común introducir sobreajuste a través de la modificación manual de los hiperparámetros de un modelo conforme vamos reportando el error de generalización sobre el conjunto de prueba.

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/train_val.svg" width="70%">

En la anterior imagen, introducimos una nueva partición, conocida como partición de "**validación**". Esta partición es resultado de tomar la partición de entrenamiento y volver a dividirla (en entrenamiento y validación) de tal forma que cualquier configuración de parámetros que se use para entrenar un modelo, pueda ser reportada en **validación**. Una vez estemos seguros que tenemos el modelo con el mejor desempeño en **validación**, volvemos a unir ambas particiones, entrenamos un modelo sobre la partición original de entrenamiento y reportamos **una sola vez** en el conjunto de prueba.

# Validación cruzada de  k pliegues

A pesar de que se introdujo una nueva partición para validar los parámetros de un modelo, se sigue usando una partición reducida para entrenar el conjunto de datos. La validación cruzada nos permite usar una mayor parte de los datos para construír el modelo y generar un estimador más robusto y con mayor capacidad de generalización. En validación cruzada, los datos son particionados varias veces en entrenamiento y validación de forma repetida. Finalmente el desempeño del clasificador es agregado sobre las diferentes particiones de validación para obtener un estimador más robusto.

La validación cruzada se hace comúnmente de la siguiente manera:
* Se divide el conjunto de entrenamiento en k-pliegues o particiones (usualmente 3, 5 o 10).
* Estas particiones deben ser del mismo tamaño
* En cada iteración uno de los pliegues es usado como la partición de validación, mientras el resto es usado como la partición de entrenamiento.
* Se reporta y guarda el desempeño sobre esa partición de validación

<img src="https://github.com/marcoteran/deeplearning/raw/master/notebooks/figures/cv2.svg" width="70%">

Vamos a usar el conjunto de datos IRIS y construímos cada uno de los pliegues:

In [ ]:
# Cargamos los datos iris
X=iris.data
y=iris.target

In [ ]:
# Creamos el clasificador LinearSVC
linSVC =LinearSVC()

In [ ]:
# Número de pliegues k=5 (mostrar tamaño de cada pliegue)
k=5
n_samples=len(X)
fold_size=n_samples/k

print("Número de muestras por pliegue: {}".format(fold_size))


Primero hacemos un reordenamiento aleatorio de los datos utilizando la función `shuffle`. De tal forma que las clases estén distribuídas a lo largo de los $k$-pliegues.

In [ ]:
from sklearn.utils import shuffle
X,y=shuffle(X,y,random_state=0)

Para cada pliegue vamos a construir una máscara sobre los datos, que indicará mi partición de entrenamiento y de validación.

In [ ]:
# Mostrar la distribución de equiquetas por pliegue
scores = []
masks = []


for fold in range(k):
    val_mask = np.zeros(n_samples, dtype=bool)
    val_mask[int(fold * fold_size) : int((fold + 1) * fold_size)] = True
    masks.append(val_mask)
    X_val, y_val = X[val_mask], y[val_mask]
    print('Distribución de etiquetas en el pliegue {}: {}'
          .format(fold, np.bincount(y_val)))
    
    X_train, y_train = X[~val_mask], y[~val_mask]
    linSVC.fit(X_train, y_train)
    scores.append(linSVC.score(X_val, y_val))

A continuación tenemos el accuracy para cada pliegue de **validación** y su respectivo promedio:

In [ ]:
# Mostramos la accuracy por cada pliegue
# Mostramos promedio sobre los 5 pliegues
print("Accuracy por cada pliegue: {}".format(scores))
print("Promedio sobre los 5 pliegues: {}".format(np.mean(scores)))

Podemos tambien observar la porción de los datos usados para cada pliegue. En negro encontramos aquel porcentaje usado para validación.

In [ ]:
# Dibujas las mascaras
pl.matshow(masks,cmap='gray_r')

## Validación cruzada usando Scikit-Learn

Scikit-Learn provee una implementación muy eficiente para realizar **validación cruzada** usando `sklearn.model_selection` [(ver más)](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html). `sklearn.model_selection.cross_val_score` recibe un estimador y un conjunto de datos, luego 
hace el particionamiento y entrena un modelo sobre cada partición de validación.

In [ ]:
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

El parámetro `cv` en `cross_val_score` controla el número de pliegues a usar.

In [ ]:
# Carga de datos iris y definición del clasificador
X=iris.data
y=iris.target

classifier=LinearSVC(max_iter=10000)

In [ ]:
classifier

In [ ]:
# Validación cruzada para k=5
scores = cross_val_score(classifier, X, y, cv=5)

In [ ]:
# Mostramos la accuracy por cada pliegue
# Mostramos promedio sobre los 5 pliegues
print("Accuracy por cada pliegue: {}".format(scores))
print("Promedio sobre los 5 pliegues: {}".format(np.mean(scores)))

`cross_val_score` realiza por defecto una partición estratificada usando `sklearn.model_selection.StratifiedKFold`. Esta estrategia consiste en hacer un particionamiento de tal forma que cada partición tenga la misma distribución de etiquetas $y$. En caso que no se quiera hacer una partición estratificada, se puede usar `sklearn.model_selection.KFold`.

`sklearn.model_selection.StratifiedKFold` genera de forma automática la partición estratificada, sin necesidad de hacer una permutación de los datos como lo hicimos anteriormente. A continuación revisamos los índices que genera `StratifiedKFold`:

In [ ]:
# Realizar la estratificación y mostrar:
# Tamaño entrenamiento:
# Tamaño validación:

cv=StratifiedKFold(n_splits=5)

In [ ]:
for train, val in cv.split(X,y):
  print("Datos de validación: {}".format(val))
  print("Tamaño de entrenamiento: {}".format(len(train)))
  print("Tamaño de validación: {}".format(len(val)))


Usando una función definida previamente, vamos a validar la distribución de las etiquetas por clase.

In [ ]:
# Utilizar plot_labels con StratifiedKFold
cv=StratifiedKFold(n_splits=5)
ylabels=[]

for train, val in cv.split(X,y):
  ylabels.append(y[val])

plot_labels(5, 3, ylabels)

Por otro lado, usando `KFold` podemos obtener un particionamiento de tantos pliegues como se especifiquen. Sin embargo, se corre el riesgo de no generar particiones balanceadas, por lo tanto lo estimadores no van a tener el desempeño esperado. Por ejemplo en el **Pliegue 0** solo hay datos de la clase $0$.

In [ ]:
# Utilizar plot_labels con KFold
cv=KFold(n_splits=5)
ylabels=[]

for train, val in cv.split(X,y):
  ylabels.append(y[val])

plot_labels(5, 3, ylabels)

# Grid search

SVM usando RBF comúnmente requiere el ajuste de dos parámetros:
* `gamma`
* `C`

Ambos parámetros pueden ser explorados usando un retículo (grid) de parámetros y evaluando su desempeño usando validación cruzada de $k$ pliegues. A continuación, creamos una partición entrenamiento y prueba sobre el conjunto de datos IRIS.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
# especificar estratificación
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                    test_size=0.3,
                                                    random_state=0)

Definimos los siguientes valores para $C=2i$ y $\gamma=i^2$, para $i=\{-5,7\}$.

In [ ]:
grid=[i for i in range(-5,7,1)]
param_grid={'C':[2**i for i in grid], 'gamma':[2**(0.5*i) for i in grid]}

Valores de $C$:

In [ ]:
param_grid['C']

Valores de $\textit{gamma}$:

In [ ]:
param_grid['gamma']

`GridSearchCV` recibe dos elementos fundamentales:
* `estimator`: Modelo de Scikit-Learn. Puede ser `SVC(kernel='rbf')`.
* `param_grid`: Diccionario que contiene los parámetros que se van a explorar usando validación cruzada.

In [ ]:
clf=GridSearchCV(SVC(kernel='rbf'),param_grid=param_grid,n_jobs=-1,verbose=2)

In [ ]:
clf

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.cv_results_

Los valores promedio de accuracy para cada combinación de hiperparámetros se pueden extraer usando `GridSearchCV.cv_results_`. Convertimos ese diccionario a un DataFrame de pandas.

In [ ]:
import pandas as pd

cv_results=pd.DataFrame(clf.cv_results_)

In [ ]:
cv_results

A continuación, observamos que el número de filas de ese DataFrame corresponde al número de configuraciones de hiperparámetros que se están explorando:

In [ ]:
print(len(cv_results))
print(len(param_grid['C']))
print(len(param_grid['gamma']))
total=len(param_grid['C'])*len(param_grid['gamma'])
print(total)

Usando la columna `mean_test_score`, extraemos los accuracy promedio de la siguiente forma:

In [ ]:
scores=clf.cv_results_['mean_test_score'].reshape(len(param_grid['C']),len(param_grid['gamma']))
scores

A continuación, presentamos una forma de visualizar esta exploración sobre la malla de hiperparámetros

In [ ]:
plt.figure(figsize=(10, 6))
plt.subplots_adjust(left=.2, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', cmap=plt.cm.hot,
           norm=MidpointNormalize(vmin=0.2, midpoint=0.92, vmax=1.))
plt.xlabel('gamma')
plt.ylabel('C')
plt.colorbar()
plt.xticks(np.arange(len(param_grid['gamma'])), param_grid['gamma'], rotation=45)
plt.yticks(np.arange(len(param_grid['C'])), param_grid['C'])
plt.title('Accuracy en validación')
plt.show()

Encontramos que existen tres modelos con el mejor desempeño usando validación cruzada:
* $C=2$ y $\gamma=0.5$
* $C=8$ y $\gamma=0.125$
* $C=4$ y $\gamma=0.5$

Para validar esta información, `GridSearchCV` nos ofrece una serie de métodos que nos permite consultar:
* La lista de resultados por elemento en la malla de parámetros (`cv_results_`)
* La configuración con el mejor desempeño (`best_params_`)
* El accuracy promediado sobre todos los pliegues de la mejor configuración (`best_score_`)

Para encontrar las mejores configuraciones, ordenamos la tabla de resultados de la siguiente manera:

In [ ]:
cv_bestresults=cv_results[['param_C','param_gamma','mean_test_score']]

In [ ]:
cv_bestresults.head()

In [ ]:
cv_bestresults.sort_values(by='mean_test_score', ascending=False)

Esta información también se puede consultar usando `.best_params_` y `.best_score_`.

In [ ]:
clf.best_score_

In [ ]:
clf.best_params_

Una vez se haya entrenado el modelo usando validación cruzada, `GridSearchCV` escoge automáticamente la mejor configuración y vuelve a entrenar un modelo sobre todo el conjunto de datos de entrenamiento. Por lo tanto se pueden hacer llamados a funciones como `predict()` y `score()`.

Para reportar sobre el conjunto de prueba basta con:

In [ ]:
clf.score(X_test,y_test)

___
¡Todo bien! ¡Es todo por hoy! 😀